<a href="https://colab.research.google.com/github/alexjl001/alexjl001.github.io/blob/master/Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
# Web Scraping for Macro
# can't call directly via pd.read_html() as it returns 403 (forbidden) --> need to do a bit of work, but still no Selenium
# https://tradingeconomics.com/united-states/indicators
import requests
import pandas as pd
from bs4 import BeautifulSoup


url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

response = requests.get(url, headers=headers)


In [77]:
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, "html.parser")

    # You need to be able to find this table tag and read all behind it
    # Find the div with class "table-responsive"
    table_div = soup.find("div", class_="mw-body-content")

    # Extract the table within the div
    table = table_div.find("table")
    data = []

    headers = [th.text.strip() for th in table.find_all("th")]

    for row in table.find_all("tr")[1:]:
      row_data = [td.text.strip() for td in row.find_all("td")]
      selected_data = [row_data[0], row_data[1], row_data[5]] #  Select 1st, 2nd and 5th columns
      data.append(selected_data)


    # Use pandas to read the table into a DataFrame

    df = pd.DataFrame(data, columns=[headers[0], headers[1], headers[5]]) # Select 1st, 2nd and 5th columns

    # df = pd.read_html(str(table))[0]  # Assuming there's only one table, otherwise, loop through the list

    # Display the DataFrame
    print(df)

    # Save to CSV (optional)
    df.to_csv("tabla_500.csv", index=False)

else:
    print("Failed to retrieve data from the webpage.")

    Symbol             Security  Date added
0      MMM                   3M  1957-03-04
1      AOS          A. O. Smith  2017-07-26
2      ABT  Abbott Laboratories  1957-03-04
3     ABBV               AbbVie  2012-12-31
4      ACN            Accenture  2011-07-06
..     ...                  ...         ...
498    XYL           Xylem Inc.  2011-11-01
499    YUM          Yum! Brands  1997-10-06
500   ZBRA   Zebra Technologies  2019-12-23
501    ZBH        Zimmer Biomet  2001-08-07
502    ZTS               Zoetis  2013-06-21

[503 rows x 3 columns]


In [78]:
# Convert 'Date' column to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date added'])

# Extract the year
df['Year'] = df['Date'].dt.year

print(df)

yearly_counts = df.groupby('Year').size().reset_index(name='count') # or .count()

print(yearly_counts)

sorted_year_counts = yearly_counts.sort_values('count', ascending=False)

print(sorted_year_counts)



    Symbol             Security  Date added       Date  Year
0      MMM                   3M  1957-03-04 1957-03-04  1957
1      AOS          A. O. Smith  2017-07-26 2017-07-26  2017
2      ABT  Abbott Laboratories  1957-03-04 1957-03-04  1957
3     ABBV               AbbVie  2012-12-31 2012-12-31  2012
4      ACN            Accenture  2011-07-06 2011-07-06  2011
..     ...                  ...         ...        ...   ...
498    XYL           Xylem Inc.  2011-11-01 2011-11-01  2011
499    YUM          Yum! Brands  1997-10-06 1997-10-06  1997
500   ZBRA   Zebra Technologies  2019-12-23 2019-12-23  2019
501    ZBH        Zimmer Biomet  2001-08-07 2001-08-07  2001
502    ZTS               Zoetis  2013-06-21 2013-06-21  2013

[503 rows x 5 columns]
    Year  count
0   1957     53
1   1964      1
2   1965      2
3   1969      2
4   1970      2
5   1972      2
6   1973      2
7   1974      1
8   1975      2
9   1976     11
10  1978      1
11  1979      2
12  1980      3
13  1981      3
14  

In [79]:
# Convert 'Date' column to datetime if it's not already

from datetime import date
today = str(date.today())

df['Date'] = pd.to_datetime(df['Date added'])

# Extract the year
df['Year'] = df['Date'].dt.year

print(today)

df['Antig'] = 2025 - df['Year']

print(df)


old_counts = df.groupby('Antig').size().reset_index(name='count2') # or .count()

print(old_counts)

sorted_old_counts = old_counts.sort_values('Antig', ascending=False)

print(sorted_old_counts)


2025-05-31
    Symbol             Security  Date added       Date  Year  Antig
0      MMM                   3M  1957-03-04 1957-03-04  1957     68
1      AOS          A. O. Smith  2017-07-26 2017-07-26  2017      8
2      ABT  Abbott Laboratories  1957-03-04 1957-03-04  1957     68
3     ABBV               AbbVie  2012-12-31 2012-12-31  2012     13
4      ACN            Accenture  2011-07-06 2011-07-06  2011     14
..     ...                  ...         ...        ...   ...    ...
498    XYL           Xylem Inc.  2011-11-01 2011-11-01  2011     14
499    YUM          Yum! Brands  1997-10-06 1997-10-06  1997     28
500   ZBRA   Zebra Technologies  2019-12-23 2019-12-23  2019      6
501    ZBH        Zimmer Biomet  2001-08-07 2001-08-07  2001     24
502    ZTS               Zoetis  2013-06-21 2013-06-21  2013     12

[503 rows x 6 columns]
    Antig  count2
0       0       5
1       1      16
2       2      15
3       3      16
4       4      15
5       5      12
6       6      22
7    